<a href="https://colab.research.google.com/github/hunkim/ACL-2020-Papers/blob/master/generate_paper_list_with_arxiv_link.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Paper List

In [0]:
def read_papers(path):
    papers = [[]]
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                papers[-1].append(line)
            else:
                papers.append([])
    for p in papers:
        assert len(p) == 2
    return papers

In [2]:
longp = read_papers("./data/long.txt")
longp[:3]

[['2kenize: Tying Subword Sequences for Chinese Script Conversion',
  'Pranav A and Isabelle Augenstein'],
 ['A Batch Normalized Inference Network Keeps the KL Vanishing Away',
  'Qile Zhu, Wei Bi, Xiaojiang Liu, Xiyao Ma, Xiaolin Li and Dapeng Wu'],
 ['A Call for More Rigor in Unsupervised Cross-lingual Learning',
  'Mikel Artetxe, Sebastian Ruder, Dani Yogatama, Gorka Labaka and Eneko Agirre']]

In [3]:
len(longp)

571

In [4]:
short = read_papers("./data/short.txt")
short[:3]

[['A Complete Shift-Reduce Chinese Discourse Parser with Robust Dynamic Oracle',
  'Shyh-Shiun Hung, Hen-Hsen Huang and Hsin-Hsi Chen'],
 ['A Diverse Corpus for Evaluating and Developing English Math Word Problem Solvers',
  'Shen-yun Miao, Chao-Chun Liang and Keh-Yih Su'],
 ['A Frame-based Sentence Representation for Machine Reading Comprehension',
  'Shaoru Guo, Ru Li, Hongye Tan, Xiaoli Li, Yong Guan, Hongyan Zhao and Yueping Zhang']]

In [5]:
len(short)

208

In [6]:
demop = read_papers("./data/demo.txt")
demop[:3]

[['ADVISER: A Toolkit for Developing Multi-modal, Multi-domain and Socially-engaged Conversational Agents',
  'Chia-Yu Li, Daniel Ortega, Dirk Väth, Florian Lux, Lindsey Vanderlyn, Maximilian Schmidt, Michael Neumann, Moritz Völkel, Pavel Denisov, Sabrina Jenne, Zorica Kacarevic and Ngoc Thang Vu'],
 ['BENTO: A Visual Platform for Building Clinical NLP Pipelines Based on CodaLab',
  'Yonghao Jin, Fei Li and Hong Yu'],
 ['Clinical-Coder: Assigning Interpretable ICD-10 Codes to Chinese Clinical Notes',
  'Pengfei Cao, Chenwei Yan, xiangling fu, Yubo Chen, Kang Liu, Jun Zhao, Shengping Liu and Weifeng Chong']]

In [7]:
len(demop)

43

In [8]:
student = read_papers("./data/student.txt")
student[:3]

[['#NotAWhore! A Computational Linguistic Perspective of Rape Culture and Victimization on Social Media',
  'Ashima Suvarna and Grusha Bhalla'],
 ['A Geometry-Inspired Attack for Generating Natural Language Adversarial Examples',
  'Zhao Meng and Roger Wattenhofer'],
 ['A Simple and Effective Dependency parser for Telugu',
  'Sneha Nallani, Manish Shrivastava and Dipti Sharma']]

In [9]:
len(student)

49

# Sorting by Topic



In [10]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk

nltk.download('wordnet')
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
  return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
  result=[]
  for token in gensim.utils.simple_preprocess(text) :
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
      result.append(lemmatize_stemming(token))
                
  return result




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
#FIXME: Better way to get human readable topic names from LDA topics?
def list2topiclist(list, num_topics = 8):
  processed_docs = []
  for line in list:
    processed_line = preprocess(line[0])
    processed_docs.append(processed_line)

    dictionary = gensim.corpora.Dictionary(processed_docs)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

  
  lda = gensim.models.LdaModel(bow_corpus, num_topics, 
                               id2word = dictionary, passes = 10)


  def get_topic_title(idx, topn=3):
    topn_terms = [dictionary[x[0]] for x in lda.get_topic_terms(idx, topn)]
    return " ".join(topn_terms)

  # Create topic title
  list_topic_titles = []
  for i in range(num_topics):
    list_topic_titles.append(get_topic_title(i))

  # Assign list to topic
  topic_dict = {}
  for line in list:
    processed_line = preprocess(line[0])
    bow_vector = dictionary.doc2bow(processed_line)
    line_topic = sorted(lda.get_document_topics(bow_vector), 
                        key=lambda tup: tup[1], reverse=True)
    topic_title = list_topic_titles[line_topic[0][0]]

    if topic_title not in topic_dict:
      topic_dict[topic_title] = []

    topic_dict[topic_title].append(line)
  
  return topic_dict
      


In [12]:
print(demop)
topics = list2topiclist(demop)
for topic in topics:
  print(topic)

[['ADVISER: A Toolkit for Developing Multi-modal, Multi-domain and Socially-engaged Conversational Agents', 'Chia-Yu Li, Daniel Ortega, Dirk Väth, Florian Lux, Lindsey Vanderlyn, Maximilian Schmidt, Michael Neumann, Moritz Völkel, Pavel Denisov, Sabrina Jenne, Zorica Kacarevic and Ngoc Thang Vu'], ['BENTO: A Visual Platform for Building Clinical NLP Pipelines Based on CodaLab', 'Yonghao Jin, Fei Li and Hong Yu'], ['Clinical-Coder: Assigning Interpretable ICD-10 Codes to Chinese Clinical Notes', 'Pengfei Cao, Chenwei Yan, xiangling fu, Yubo Chen, Kang Liu, Jun Zhao, Shengping Liu and Weifeng Chong'], ['CLIReval: Evaluating Machine Translation as a Cross-Lingual Information Retrieval Task', 'Shuo Sun, Suzanna Sia and Kevin Duh'], ['Conversation Learner - A Machine Teaching Tool for Building Dialog Managers for Task-Oriented Dialog Systems', 'Swadheen Shukla, Lars Liden, Shahin Shayandeh, Eslam Kamal, Jinchao Li, Matt Mazzola, Thomas Park, Baolin Peng and Jianfeng Gao'], ['ConvLab-2: An O

# Search arXiv Link

In [0]:
from googlesearch import search
import urllib
from bs4 import BeautifulSoup
from difflib import SequenceMatcher
from tqdm import tqdm
import time

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()


def search_arxiv_link(title):
    link = None
    for j in search(title, tld="co.in", num=10, stop=1, pause=1.0, user_agent="acl2020"):
        if 'arxiv.org/abs' in j:
            thepage = urllib.request.urlopen(j)
            soup = BeautifulSoup(thepage, "html.parser")
            searched_title = ' '.join(soup.title.text.lower().split()[1:])
            if similarity(title, searched_title) > 0.8:
                link = j
                break
            else:
                print("NOT MATCHED")
                print(title)
                print(searched_title)
    return link

In [0]:
def generate_paper_list_with_arxiv_link(f, papers):
    for p in tqdm(papers):
        title, authors = p
        link = search_arxiv_link(title.lower())
        if link:
            f.write(f"- {title} [[arXiv]]({link})\n")
        else:
            f.write(f"- {title}\n")
    f.write("\n")

In [0]:
def generate_paper_list_with_arxiv_link_topic(f, papers):
  topic_papers = list2topiclist(papers)
  for topic in topic_papers:
    f.write("### " + topic + "\n")
    generate_paper_list_with_arxiv_link(f, topic_papers[topic])


In [19]:
with open("papers_with_arxiv_link_topic.md", "w") as f:
  f.write("## Long Papers\n\n")
  generate_paper_list_with_arxiv_link_topic(f, longp)
  f.write("## Short Papers\n\n")
  generate_paper_list_with_arxiv_link_topic(f, short)
  f.write("## System Demonstrations\n\n")
  generate_paper_list_with_arxiv_link_topic(f, demop)
  f.write("## Student Research Workshop\n\n")
  generate_paper_list_with_arxiv_link_topic(f, student)



 97%|█████████▋| 67/69 [02:26<00:04,  2.47s/it]

NOT MATCHED
zero-shot text classification via reinforced self-training
integrating semantic knowledge to tackle zero-shot text classification



100%|██████████| 69/69 [02:30<00:00,  2.18s/it]

 11%|█         | 8/73 [00:17<02:22,  2.19s/it]

NOT MATCHED
attentive pooling with learnable norms for text representation
attentive pooling networks



 23%|██▎       | 17/73 [00:37<02:04,  2.23s/it]

...


100%|██████████| 8/8 [00:17<00:00,  2.20s/it]

100%|██████████| 5/5 [00:10<00:00,  2.16s/it]

100%|██████████| 6/6 [00:12<00:00,  2.15s/it]

  0%|          | 0/5 [00:00<?, ?it/s]

NOT MATCHED
unsupervised multilingual sentence embeddings for parallel corpus mining
margin-based parallel corpus mining with multilingual sentence embeddings



100%|██████████| 5/5 [00:11<00:00,  2.29s/it]

 75%|███████▌  | 6/8 [00:14<00:05,  2.65s/it]

NOT MATCHED
υbleu: uncertainty-aware automatic evaluation method for open-domain dialogue systems
better automatic evaluation of open-domain dialogue systems with contextualized embeddings



100%|██████████| 8/8 [00:19<00:00,  2.41s/it]

100%|██████████| 3/3 [00:06<00:00,  2.11s/it]

100%|██████████| 7/7 [00:15<00:00,  2.23s/it]

100%|██████████| 7/7 [00:16<00:00,  2.34s/it]


In [0]:
with open("papers_with_arxiv_link.md", "w") as f:
    f.write("## Long Papers\n\n")
    generate_paper_list_with_arxiv_link(f, longp)
    f.write("## Short Papers\n\n")
    generate_paper_list_with_arxiv_link(f, short)
    f.write("## System Demonstrations\n\n")
    generate_paper_list_with_arxiv_link(f, demop)
    f.write("## Student Research Workshop\n\n")
    generate_paper_list_with_arxiv_link(f, student)